# CS109A Final Project: Data Driven March Madness 

**Harvard University**<br>
**Fall 2016**<br>
**Authors: Kurt Bullard and Kendrick Vinar**<br>

# Project Walkthrough

Below is a detailed examination of the process by which we created our models. The table of contents below can be used to navigate to a topic of interest. 
_____________________

- [Data Sources](#data_sources)
- [Data Cleaning](#data_cleaning)
- [Variable Creation/Selection](#variable_creation)
- [2016 Data Extraction](#2016_data)
- [Preparing the Data for Analysis](#preparation)

Return back to the [homepage](home_page.html).

<a id = 'data_sources'></a>

## Data Sources

We retrieved data from two sources: the [2016 Kaggle March Madness Learning Mania](https://www.kaggle.com/c/march-machine-learning-mania-2016) competition and the [Pomeroy College Basketball Rankings](http://www.kenpom.com). The Kaggle dataset provided us with detailed information for each team’s regular season performance in addition to results from that year’s tournament. The Pomeroy rankings—or KenPom, as they are colloquially known— give additional advanced metrics for each team, including but not limited to offensive efficiency, defensive efficiency, and adjusted tempo. 

We imported nine CSV files from these sources, including information about seeding, and box scores of tournament and regular season games from 2003-2016

<a id = 'data_cleaning'></a>

## Data Cleaning

While sorting through the data, we realized we needed an easy way to distinguish between data for a Duke team that played in 2005 and a Duke team that played in 2010. For this, we created a unique ID for each team that would identify both the team and the year by combining the team name and the season in which it played. 

We merged the KenPom data into the detailed tournament dataset by choosing the variables we wanted to consider for our model. These included rankings such as offensive and defensive efficiency, advanced metrics that were not included in the Kaggle data. 

<a id = 'variable_creation'></a>

## Variable Creation and Selection

We created a variable for tournament seedings [1, 16] to normalize seed across regions. We did this by scraping the integers from the seed column that also referenced the region from which the bracket came. 

We created new variables with data given us from the Kaggle dataset that we felt would be better predictors. For example, we calculated predictors such as assist ratio, which is calculated as the number of assists in a game divided by the number of made field goals in a game. This predictor functions as a proxy for how team-oriented a team is or how reliant they are on star players. We created other variables in a similar fashion, including offensive and defensive rebound ratio, point differential, and the percentage of shots in the game that were two-point field goals and three-point field goals. 

We also created two variables to quantify performance against other teams in the NCAA tournament. One variable represents wins against other teams in the tournament and another variable represents losses against other teams in the tournament. For a team that beats a tournament ranked $n$ in the tournament, we gave them $1/n$ points. This system weights wins against highly-ranked teams in the tournament much more valuable than wins against lowly-ranked teams. This technique for predictor creation was shown to be effective by John Ezekowitz in his paper, [insert link]. 

[kurt can you say more here about what happens next it’s getting pretty fuzzy for me here]

Once we aggregated all the predictors together, we created a correlation matrix to determine which predictors were correlated with each other. We checked for multicollinearity with variance inflation factor (VIF) and eliminated [kurt can you say more here?]

[kurt can you also say something about the interaction terms]

After selecting the desired variables, we created a new data frame to hold our selected predictors. We found that a lot of this data manipulation was very time intensive, so we chose to separate this into a distinct notebook that could be run only when changes were made. At the end of this notebook, we export a new CSV file that serves as the starting point for our data analysis in another notebook.



<a id = '2016_data'></a>

## 2016 Data Extraction

[kurt, can you say something about the 2016 data and how that was different?] 


<a id = 'preparation'></a>

## Preparing the Data for Analysis

